<a href="https://colab.research.google.com/github/AndreiAf02/Cholera-Data/blob/main/STAT561_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [152]:
import matplotlib.pyplot as plt
import numpy as np
import pymc as pm
import seaborn as sns
import pandas as pd

In [153]:
path_to_data = 'https://raw.githubusercontent.com/AndreiAf02/STAT561_Project/main/'

According to the documentation for the European Union Statistics on Income and Living Conditions (EU-SILC) given [here](https://ec.europa.eu/eurostat/documents/203647/16195750/2021_Doc65_EUSILC_User_Guide.pdf), the statistics of interest are defined as follows:
* **DB010**: Survey Year (taken to be 2013, the most recent year available)
* **DB020**: Residence Country
* **DB030**: Household ID (unique identifier)
* **DB040**: Region of Residence (used in our study for within-country stratification of the sample)
* **HB030**: Household ID (identical to DB030 above)
* **HH021**: Tenure Status (1 - *Owner w/o mortgage* ; 2 - *Owner w/ mortgage* ; 3 - *Tenant at market price* ; 4 - *Tenant at reduced price* ; 5 - *Tenant at market price*)
* **HH030**: Number of available rooms (maximum value of 10) **TO DISCUSS THAT COULD LEAD TO MEASUREMENT ERROR!**
* **HH050**: Able to warm home adequately (1 - *Yes* ; 2 - *No*)
* **HH070**: Total Housing Cost (cost per month, including mortgage interest payments, utilities, etc.)
* **HH071**: Mortgage Principal Repayment (cost per month)
* **HS050**: Afford meat (incl. fish, chicken, or vegetarian equivalent) every other day (1 - *Yes* ; 2 - *No*)
* **HS060**: Able to cover unexpected expenses (1 - *Yes* ; 2 - *No*)
* **HS120**: Able to make ends meet (1 - *great difficulty* ;  2 - *difficulty* ; 3 - *some difficulty* ; 4 - *fairly easily* ; 5 - *easily* ; 6 - *very easily*)




In this case, we define the function for calculating the sample variances for the strata in each country (where stratification is present) below:

In [149]:
strat_results = pd.DataFrame(columns=['y_bar', 'Var_ybar'])

def strat(data, label):
      country = []
      region = []
      size = []
      weight = []

      strat_mean = []

      strat_var = []

      strat_var_ybar = []

      Region_list = data['DB040'].unique()

      stratif = pd.DataFrame(columns=['Region', 'Size', 'Weight'])

      for i in range(len(Region_list)):
          data_1 = data[data['DB040'] == Region_list[i]]
          data_len = len(data_1)
          wt_1 = data_len/size_total

          mean_st = np.mean(data_1[label])

          # if label=='HH050' or label=='HS050' or label=='HS060':
          if label in ('HH050', 'HS050', 'HS060'):  ## Calculating mean and variance for proportion
            mean_st = 2 - np.mean(data_1[label])
            var_st = data_len/(data_len-1)*mean_st*(1-mean_st)
          else:
            mean_st = np.mean(data_1[label])
            var_st = np.var(data_1[label], ddof=1)

          var_ybar = var_st/data_len  ## We ignore fpc, since stratum population is unknown but very large

          country.append(data_1['DB020'].unique())
          region.append(Region_list[i])
          size.append(data_len)
          weight.append(wt_1)
          strat_mean.append(mean_st)
          strat_var.append(var_st)
          strat_var_ybar.append(var_ybar)
      stratif = pd.DataFrame([country, region, size, weight, strat_mean, strat_var, strat_var_ybar]).T
      stratif.columns =['Country', 'Region', 'Size', 'Weight', 'Mean of '+str(label), 'var of '+str(label), 'var(y_bar)']
      print(stratif)
      weighted_mean = sum(stratif['Weight']*stratif['Mean of '+str(label)])
      print('Weighted mean (stratified):', weighted_mean)
      weighted_var_ybar = sum(stratif['Weight']**2*stratif['var(y_bar)'])
      print('Weighted variance of sample mean (stratified):', weighted_var_ybar)
      strat_results = pd.DataFrame([weighted_mean, weighted_var_ybar])



##Austria:

In [182]:
data_AT = pd.read_csv(path_to_data + "AT_Data.csv", sep=",", header=0)

data = data_AT
data = data[data['DB010'].notnull()]
data['HH070'] = data['HH070'].replace(np.nan, 0)
data['HH071'] = data['HH071'].replace(np.nan, 0)
data['HH070+']=data['HH070']+data['HH071']
data_AT = data
data_AT

,DB010,DB020,DB030,DB040,HB030,HH021,HH030,HH050,HH070,HH071,HS050,HS060,HS120,HH070+
0,2013,AT,295,AT1,295,3,3,1,238.73,707.142500,1,1.0,6.0,945.872500
1,2013,AT,524,AT1,524,1,2,1,501.38,0.000000,1,1.0,3.0,501.380000
2,2013,AT,1693,AT1,1693,3,3,1,529.67,130.952500,1,2.0,2.0,660.622500
3,2013,AT,2979,AT1,2979,1,2,1,341.75,0.000000,1,2.0,3.0,341.750000
4,2013,AT,3173,AT1,3173,3,3,1,47.48,0.000000,1,1.0,4.0,47.480000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5972,2013,AT,3699379,AT3,3699379,3,1,1,272.59,0.000000,1,1.0,4.0,272.590000
5973,2013,AT,3700053,AT3,3700053,3,5,1,257.67,316.666667,1,1.0,5.0,574.336667
5974,2013,AT,3700232,AT3,3700232,4,5,1,790.50,533.333333,1,1.0,3.0,1323.833333
5975,2013,AT,3700410,AT3,3700410,3,6,1,402.94,135.341667,1,2.0,3.0,538.281667


In [202]:
## Ordinary estimator:

data=data_AT
label='HH050'

country = []
region = []
size = []
weight = []

strat_mean = []

strat_var = []

strat_var_ybar = []

Region_list = data['DB040'].unique()

stratif = pd.DataFrame(columns=['Region', 'Size', 'Weight'])

data_len = len(data)

if label in ('HH050', 'HS050', 'HS060'):  ## Calculating mean and variance for proportion estimator
  mean = 2 - np.mean(data[label])
  var = data_len/(data_len-1)*mean_st*(1-mean_st)
else:
  mean = np.mean(data[label])
  var = np.var(data[label], ddof=1)

ord_var_ybar = var/data_len  ## We ignore fpc, since stratum population is unknown but very large


print('Sample mean (ordinary):', mean)
print('Sample variance:', var)
print('Estimated variance of sample mean (ordinary):', ord_var_ybar)

Sample mean (ordinary): 0.965701857118956
Sample variance: 0.03273269044221854
Estimated variance of sample mean (ordinary): 5.4764414325277804e-06


In [203]:
## Stratified sampling by region:

data=data_AT
label='HH050'

country = []
region = []
size = []
weight = []

strat_mean = []

strat_var = []

strat_var_ybar = []

Region_list = data['DB040'].unique()

stratif = pd.DataFrame(columns=['Region', 'Size', 'Weight'])

for i in range(len(Region_list)):
    data_1 = data[data['DB040'] == Region_list[i]]
    data_len_1 = len(data_1)
    wt_1 = data_len_1/size_total

    # mean_st = np.mean(data_1[label])

    if label in ('HH050', 'HS050', 'HS060'):  ## Calculating mean and variance for proportion estimator
      mean_st = 2 - np.mean(data_1[label])
      var_st = data_len_1/(data_len_1-1)*mean_st*(1-mean_st)
    else:
      mean_st = np.mean(data_1[label])
      var_st = np.var(data_1[label], ddof=1)

    var_ybar = var_st/data_len_1  ## We ignore fpc, since stratum population is unknown but very large

    country.append(data_1['DB020'].unique())
    region.append(Region_list[i])
    size.append(data_len_1)
    weight.append(wt_1)
    strat_mean.append(mean_st)
    strat_var.append(var_st)
    strat_var_ybar.append(var_ybar)
stratif = pd.DataFrame([country, region, size, weight, strat_mean, strat_var, strat_var_ybar]).T
stratif.columns =['Country', 'Region', 'Size', 'Weight', 'Mean of '+str(label), 'var of '+str(label), 'var(y_bar)']
print(stratif)
weighted_mean = sum(stratif['Weight']*stratif['Mean of '+str(label)])
print('Weighted mean (stratified):', weighted_mean)
weighted_var_ybar = sum(stratif['Weight']**2*stratif['var(y_bar)'])
print('Weighted variance of sample mean (stratified):', weighted_var_ybar)

  Country Region  Size    Weight Mean of HH050 var of HH050 var(y_bar)
0    [AT]    AT1  2618  0.438012      0.964095     0.034629   0.000013
1    [AT]    AT2  1204  0.201439      0.968439     0.030591   0.000025
2    [AT]    AT3  2155  0.360549      0.966125     0.032742   0.000015
Weighted mean (stratified): 0.965701857118956
Weighted variance of sample mean (stratified): 5.543828935913479e-06


As seen below, the stratified variance is slightly larger than the ordinary estimator variance, indicating that the stratified samples are not homogeneous. Hence, we use the ordinary estimator for the variance.

In [204]:
weighted_mean_AT = weighted_mean
weighted_var_ybar_AT = weighted_var_ybar
ordinary_var_ybar_AT = ord_var_ybar
ord_var_AT = ord_var_ybar * data_len  ## Using the inverse of the var(y_bar) equation ignoring fpc

print('Weighted mean:', weighted_mean_AT)
print('Weighted var(y_bar):', weighted_var_ybar_AT)
print('Ordinary var(y_bar):', ordinary_var_ybar_AT)

print('Sample variance s^2:', ord_var_AT)


Weighted mean: 0.965701857118956
Weighted var(y_bar): 5.543828935913479e-06
Ordinary var(y_bar): 5.4764414325277804e-06
Sample variance s^2: 0.03273269044221854


##Belgium:

In [158]:
data_BE = pd.read_csv(path_to_data + "BE_Data.csv", sep=",", header=0)

data = data_BE
data = data[data['DB010'].notnull()]
data['HH070'] = data['HH070'].replace(np.nan, 0)
data['HH071'] = data['HH071'].replace(np.nan, 0)
data['HH070+']=data['HH070']+data['HH071']
data_BE = data
data_BE

,DB010,DB020,DB030,DB040,HB030,HH021,HH030,HH050,HH070,HH071,HS050,HS060,HS120,HH070+
0,2013,BE,22,BE10,22,3,6,1,100.00,0.0,1,1.0,5,100.00
1,2013,BE,28,BE10,28,1,6,2,374.00,0.0,1,1.0,5,374.00
2,2013,BE,37,BE10,37,3,5,1,605.50,0.0,2,1.0,3,605.50
3,2013,BE,43,BE10,43,3,3,1,45.66,0.0,1,1.0,4,45.66
4,2013,BE,50,BE10,50,1,6,1,427.82,0.0,1,1.0,6,427.82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6154,2013,BE,95716,BE35,95716,1,6,1,427.82,110.0,1,1.0,6,537.82
6155,2013,BE,95722,BE35,95722,2,6,1,783.88,939.0,1,1.0,6,1722.88
6156,2013,BE,95723,BE35,95723,2,5,1,449.33,1200.0,1,1.0,2,1649.33
6157,2013,BE,95821,BE35,95821,1,4,1,774.67,803.0,1,1.0,4,1577.67


##Bulgaria:

In [159]:
data_BG = pd.read_csv(path_to_data + "BG_Data.csv", sep=",", header=0)
data = data_BG
data = data[data['DB010'].notnull()]
data['HH070'] = data['HH070'].replace(np.nan, 0)
data['HH071'] = data['HH071'].replace(np.nan, 0)
data['HH070+']=data['HH070']+data['HH071']
data_BG = data
data_BG


,DB010,DB020,DB030,DB040,HB030,HH021,HH030,HH050,HH070,HH071,HS050,HS060,HS120,HH070+
0,2013,BG,31,BG31,31,1,3,2,50.11,0.0,2,2,4,50.11
1,2013,BG,41,BG31,41,1,3,2,54.20,0.0,1,2,2,54.20
2,2013,BG,53,BG31,53,1,2,1,86.92,0.0,2,2,1,86.92
3,2013,BG,63,BG31,63,1,3,2,196.34,0.0,1,1,3,196.34
4,2013,BG,72,BG31,72,1,2,1,112.49,0.0,2,2,1,112.49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4966,2013,BG,133242,BG42,133242,1,2,1,91.61,0.0,1,2,3,91.61
4967,2013,BG,133246,BG42,133246,1,4,2,125.27,0.0,1,1,2,125.27
4968,2013,BG,133249,BG42,133249,1,4,2,135.49,0.0,1,1,5,135.49
4969,2013,BG,133251,BG42,133251,1,6,1,66.47,0.0,2,2,4,66.47


##Cyprus:

In [160]:
data_CY = pd.read_csv(path_to_data + "CY_Data.csv", sep=",", header=0)
data = data_CY
data = data[data['DB010'].notnull()]
data['HH070'] = data['HH070'].replace(np.nan, 0)
data['HH071'] = data['HH071'].replace(np.nan, 0)
data['HH070+']=data['HH070']+data['HH071']
data_CY = data
data_CY

,DB010,DB020,DB030,DB040,HB030,HH021,HH030,HH050,HH070,HH071,HS050,HS060,HS120,HH070+
0,2013,CY,4,CY00,4,3,6,1,164.50,0.00,1,2,1,164.50
1,2013,CY,25,CY00,25,2,6,1,235.00,0.00,2,1,2,235.00
2,2013,CY,41,CY00,41,3,6,1,180.83,0.00,1,1,1,180.83
3,2013,CY,49,CY00,49,1,5,2,288.83,526.25,1,2,1,815.08
4,2013,CY,54,CY00,54,1,2,1,29.17,119.58,2,2,2,148.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4643,2013,CY,62043,CY00,62043,2,4,1,1842.50,780.17,1,1,4,2622.67
4644,2013,CY,62075,CY00,62075,2,6,1,243.33,0.00,1,2,1,243.33
4645,2013,CY,62129,CY00,62129,1,4,1,964.17,2773.75,1,1,6,3737.92
4646,2013,CY,62147,CY00,62147,1,6,1,700.75,0.00,1,1,6,700.75


##Germany:

In [161]:
data_DE = pd.read_csv(path_to_data + "DE_Data.csv", sep=",", header=0)
data = data_DE
data = data[data['DB010'].notnull()]
data['HH070'] = data['HH070'].replace(np.nan, 0)
data['HH071'] = data['HH071'].replace(np.nan, 0)
data['HH070+']=data['HH070']+data['HH071']
data_DE = data
data_DE

,DB010,DB020,DB030,DB040,HB030,HH021,HH030,HH050,HH070,HH071,HS050,HS060,HS120,HH070+
0,2013,DE,356,NaN,356,5,3,1.0,459.00,105.0,1.0,1.0,5.0,564.00
1,2013,DE,447,NaN,447,1,6,1.0,969.17,0.0,1.0,2.0,5.0,969.17
2,2013,DE,599,NaN,599,3,4,1.0,510.00,0.0,1.0,1.0,5.0,510.00
3,2013,DE,711,NaN,711,2,6,1.0,590.00,0.0,2.0,2.0,4.0,590.00
4,2013,DE,795,NaN,795,2,5,1.0,1140.67,0.0,1.0,1.0,6.0,1140.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12698,2013,DE,4010281,NaN,4010281,1,3,1.0,420.00,1900.0,1.0,2.0,3.0,2320.00
12699,2013,DE,4010304,NaN,4010304,2,5,1.0,0.00,274.0,1.0,1.0,3.0,274.00
12700,2013,DE,4010309,NaN,4010309,3,3,1.0,696.67,0.0,1.0,1.0,2.0,696.67
12701,2013,DE,4010422,NaN,4010422,2,3,2.0,380.00,0.0,1.0,1.0,5.0,380.00


##Denmark:

In [162]:
data_DK = pd.read_csv(path_to_data + "DK_Data.csv", sep=",", header=0)
data = data_DK
data = data[data['DB010'].notnull()]
data['HH070'] = data['HH070'].replace(np.nan, 0)
data['HH071'] = data['HH071'].replace(np.nan, 0)
data['HH070+']=data['HH070']+data['HH071']
data_DK = data
data_DK

,DB010,DB020,DB030,DB040,HB030,HH021,HH030,HH050,HH070,HH071,HS050,HS060,HS120,HH070+
0,2013,DK,83,DK01,83,1,6.0,1,2234.57,0.0,1,1.0,5,2234.57
1,2013,DK,105,DK01,105,1,6.0,1,705.42,0.0,1,1.0,6,705.42
2,2013,DK,139,DK01,139,2,6.0,1,2343.81,0.0,1,1.0,4,2343.81
3,2013,DK,162,DK01,162,2,6.0,1,978.67,0.0,1,1.0,4,978.67
4,2013,DK,165,DK01,165,2,6.0,1,1526.28,0.0,1,1.0,6,1526.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5414,2013,DK,140226,DK05,140226,2,2.0,1,3057.16,0.0,1,1.0,6,3057.16
5415,2013,DK,140311,DK05,140311,1,6.0,1,793.90,0.0,1,1.0,6,793.90
5416,2013,DK,140321,DK05,140321,1,6.0,1,814.03,0.0,1,1.0,4,814.03
5417,2013,DK,140332,DK05,140332,3,2.0,1,610.98,0.0,1,2.0,4,610.98


##Estonia:

In [163]:
data_EE = pd.read_csv(path_to_data + "EE_Data.csv", sep=",", header=0)
data = data_EE
data = data[data['DB010'].notnull()]
data['HH070'] = data['HH070'].replace(np.nan, 0)
data['HH071'] = data['HH071'].replace(np.nan, 0)
data['HH070+']=data['HH070']+data['HH071']
data_EE = data
data_EE

,DB010,DB020,DB030,DB040,HB030,HH021,HH030,HH050,HH070,HH071,HS050,HS060,HS120,HH070+
0,2013,EE,7,EE00,7,5,5.0,1,85.32,0.0,1,2.0,3,85.32
1,2013,EE,24,EE00,24,1,6.0,1,50.00,0.0,1,2.0,2,50.00
2,2013,EE,49,EE00,49,2,3.0,1,381.99,0.0,1,1.0,5,381.99
3,2013,EE,52,EE00,52,2,6.0,1,107.00,0.0,1,1.0,5,107.00
4,2013,EE,79,EE00,79,5,2.0,1,77.30,0.0,1,1.0,2,77.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5770,2013,EE,117876,EE00,117876,2,3.0,1,159.00,316.0,1,1.0,5,475.00
5771,2013,EE,117880,EE00,117880,2,6.0,1,289.00,0.0,1,2.0,3,289.00
5772,2013,EE,117932,EE00,117932,1,6.0,1,229.50,0.0,1,2.0,4,229.50
5773,2013,EE,117954,EE00,117954,2,4.0,1,246.50,0.0,1,1.0,4,246.50


##Greece:

In [164]:
data_EL = pd.read_csv(path_to_data + "EL_Data.csv", sep=",", header=0)
data = data_EL
data = data[data['DB010'].notnull()]
data['HH070'] = data['HH070'].replace(np.nan, 0)
data['HH071'] = data['HH071'].replace(np.nan, 0)
data['HH070+']=data['HH070']+data['HH071']
data_EL = data
data_EL

,DB010,DB020,DB030,DB040,HB030,HH021,HH030,HH050,HH070,HH071,HS050,HS060,HS120,HH070+
0,2013,EL,60,EL30,60,5,3,1,400.05,0.00,1,1,1,400.05
1,2013,EL,65,EL30,65,1,4,1,398.78,0.00,1,1,1,398.78
2,2013,EL,82,EL30,82,1,4,2,796.29,613.51,1,1,3,1409.80
3,2013,EL,109,EL30,109,3,5,1,281.94,0.00,1,1,3,281.94
4,2013,EL,122,EL30,122,1,3,1,414.02,0.00,2,1,5,414.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7434,2013,EL,84911,EL65,84911,1,4,2,622.30,0.00,1,2,1,622.30
7435,2013,EL,84912,EL65,84912,1,4,1,400.05,0.00,1,1,1,400.05
7436,2013,EL,84914,EL65,84914,1,5,1,867.41,212.60,1,1,6,1080.01
7437,2013,EL,84924,EL65,84924,1,5,1,750.57,0.00,1,1,5,750.57


##Spain:

In [165]:
data_ES = pd.read_csv(path_to_data + "ES_Data.csv", sep=",", header=0)
data = data_ES
data = data[data['DB010'].notnull()]
data['HH070'] = data['HH070'].replace(np.nan, 0)
data['HH071'] = data['HH071'].replace(np.nan, 0)
data['HH070+']=data['HH070']+data['HH071']
data_ES = data
data_ES

,DB010,DB020,DB030,DB040,HB030,HH021,HH030,HH050,HH070,HH071,HS050,HS060,HS120,HH070+
0,2013,ES,43,ES11,43,1,6,1,1364.99,0.00,1,1,4,1364.99
1,2013,ES,55,ES11,55,3,4,1,136.14,229.00,1,2,4,365.14
2,2013,ES,72,ES11,72,1,6,1,1384.39,0.00,1,1,3,1384.39
3,2013,ES,85,ES11,85,4,6,1,269.06,127.28,1,2,5,396.34
4,2013,ES,97,ES11,97,2,4,1,164.87,0.00,1,1,4,164.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12134,2013,ES,363504,ES70,363504,1,6,1,111.77,0.00,1,1,4,111.77
12135,2013,ES,363592,ES70,363592,3,5,1,933.64,0.00,1,1,6,933.64
12136,2013,ES,363713,ES70,363713,2,3,1,150.52,0.00,1,2,3,150.52
12137,2013,ES,363723,ES70,363723,3,4,1,188.04,0.00,1,1,5,188.04


##Finland:

In [166]:
data_FI = pd.read_csv(path_to_data + "FI_Data.csv", sep=",", header=0)
data = data_FI
data = data[data['DB010'].notnull()]
data['HH070'] = data['HH070'].replace(np.nan, 0)
data['HH071'] = data['HH071'].replace(np.nan, 0)
data['HH070+']=data['HH070']+data['HH071']
data_FI = data
data_FI

,DB010,DB020,DB030,DB040,HB030,HH021,HH030,HH050,HH070,HH071,HS050,HS060,HS120,HH070+
0,2013,FI,142,FI19,142,3,1.0,1.0,449,0.00,1.0,1.0,5.0,449.00
1,2013,FI,300,FI19,300,1,2.0,2.0,244,0.00,1.0,2.0,6.0,244.00
2,2013,FI,315,FI19,315,2,3.0,1.0,318,0.00,1.0,1.0,4.0,318.00
3,2013,FI,354,FI19,354,2,5.0,1.0,412,0.00,1.0,1.0,6.0,412.00
4,2013,FI,862,FI19,862,2,4.0,1.0,358,0.00,1.0,1.0,4.0,358.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11365,2013,FI,2593649,FI1D,2593649,4,5.0,1.0,242,252.50,1.0,1.0,6.0,494.50
11366,2013,FI,2594037,FI1D,2594037,2,4.0,1.0,364,326.33,1.0,2.0,6.0,690.33
11367,2013,FI,2594214,FI1D,2594214,2,2.0,1.0,504,0.00,1.0,1.0,5.0,504.00
11368,2013,FI,2594542,FI1D,2594542,1,3.0,1.0,816,804.75,1.0,1.0,6.0,1620.75


##France:

In [167]:
data_FR = pd.read_csv(path_to_data + "FR_Data.csv", sep=",", header=0)
data = data_FR
data = data[data['DB010'].notnull()]
data['HH070'] = data['HH070'].replace(np.nan, 0)
data['HH071'] = data['HH071'].replace(np.nan, 0)
data['HH070+']=data['HH070']+data['HH071']
data_FR = data
data_FR

,DB010,DB020,DB030,DB040,HB030,HH021,HH030,HH050,HH070,HH071,HS050,HS060,HS120,HH070+
0,2013,FR,211,FR10,211,2,5.0,1.0,1327.50,0.00,1.0,1.0,3.0,1327.50
1,2013,FR,265,FR10,265,2,6.0,1.0,540.92,0.00,1.0,1.0,2.0,540.92
2,2013,FR,354,FR10,354,3,6.0,1.0,310.75,0.00,1.0,2.0,4.0,310.75
3,2013,FR,420,FR10,420,2,4.0,1.0,393.92,0.00,1.0,1.0,4.0,393.92
4,2013,FR,471,FR10,471,5,3.0,1.0,752.17,0.00,1.0,1.0,3.0,752.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11126,2013,FR,557144,FR83,557144,1,4.0,1.0,575.00,505.25,1.0,1.0,3.0,1080.25
11127,2013,FR,557146,FR83,557146,1,2.0,1.0,182.83,277.67,1.0,1.0,5.0,460.50
11128,2013,FR,557162,FR83,557162,2,3.0,1.0,258.50,1264.75,1.0,2.0,3.0,1523.25
11129,2013,FR,557197,FR83,557197,3,4.0,1.0,808.83,590.08,1.0,1.0,4.0,1398.91


##Croatia:

In [168]:
data_HR = pd.read_csv(path_to_data + "HR_Data.csv", sep=",", header=0)
data = data_HR
data = data[data['DB010'].notnull()]
data['HH070'] = data['HH070'].replace(np.nan, 0)
data['HH071'] = data['HH071'].replace(np.nan, 0)
data['HH070+']=data['HH070']+data['HH071']
data_HR = data
data_HR

,DB010,DB020,DB030,DB040,HB030,HH021,HH030,HH050,HH070,HH071,HS050,HS060,HS120,HH070+
0,2013,HR,52,HR03,52,5.0,3.0,2.0,183.85,0.0,1.0,2.0,1.0,183.85
1,2013,HR,66,HR03,66,1.0,6.0,1.0,822.07,0.0,1.0,1.0,1.0,822.07
2,2013,HR,87,HR03,87,1.0,2.0,1.0,206.74,0.0,1.0,1.0,3.0,206.74
3,2013,HR,100,HR03,100,1.0,5.0,1.0,265.45,0.0,1.0,2.0,3.0,265.45
4,2013,HR,103,HR03,103,1.0,3.0,1.0,183.91,0.0,1.0,1.0,2.0,183.91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5360,2013,HR,75894,HR04,75894,4.0,4.0,1.0,124.97,0.0,1.0,2.0,3.0,124.97
5361,2013,HR,75906,HR04,75906,1.0,4.0,1.0,150.90,0.0,1.0,2.0,3.0,150.90
5362,2013,HR,75914,HR04,75914,1.0,3.0,2.0,132.51,0.0,2.0,1.0,2.0,132.51
5363,2013,HR,75920,HR04,75920,NaN,NaN,NaN,0.00,0.0,NaN,NaN,NaN,0.00


##Hungary:

In [169]:
data_HU = pd.read_csv(path_to_data + "HU_Data.csv", sep=",", header=0)
data = data_HU
data = data[data['DB010'].notnull()]
data['HH070'] = data['HH070'].replace(np.nan, 0)
data['HH071'] = data['HH071'].replace(np.nan, 0)
data['HH070+']=data['HH070']+data['HH071']
data_HU = data
data_HU

,DB010,DB020,DB030,DB040,HB030,HH021,HH030,HH050,HH070,HH071,HS050,HS060,HS120,HH070+
0,2013,HU,115,HU1,115,1,2,1,68.97,0.00,1.0,2,2,68.97
1,2013,HU,121,HU1,121,1,2,1,111.66,0.00,1.0,2,1,111.66
2,2013,HU,152,HU1,152,1,3,2,99.97,0.00,1.0,2,1,99.97
3,2013,HU,203,HU1,203,1,3,1,93.92,11.43,1.0,2,3,105.35
4,2013,HU,234,HU1,234,1,3,1,78.65,0.00,1.0,2,2,78.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10218,2013,HU,188351,HU3,188351,2,3,1,237.97,0.00,1.0,2,4,237.97
10219,2013,HU,188362,HU3,188362,5,3,1,93.34,0.00,1.0,2,2,93.34
10220,2013,HU,188397,HU3,188397,1,6,1,192.40,0.00,1.0,2,1,192.40
10221,2013,HU,188406,HU3,188406,2,3,1,203.98,0.00,2.0,1,1,203.98


##Ireland:

In [170]:
data_IE = pd.read_csv(path_to_data + "IE_Data.csv", sep=",", header=0)
data = data_IE
data = data[data['DB010'].notnull()]
data['HH070'] = data['HH070'].replace(np.nan, 0)
data['HH071'] = data['HH071'].replace(np.nan, 0)
data['HH070+']=data['HH070']+data['HH071']
data_IE = data
data_IE

,DB010,DB020,DB030,DB040,HB030,HH021,HH030,HH050,HH070,HH071,HS050,HS060,HS120,HH070+
0,2013,IE,47,IE01,47,3,4,2,265.50,0.00,1,1,4,265.50
1,2013,IE,60,IE01,60,2,5,1,263.64,0.00,1,2,4,263.64
2,2013,IE,79,IE01,79,1,4,1,294.86,548.83,1,2,2,843.69
3,2013,IE,93,IE01,93,3,6,1,1201.81,0.00,1,2,2,1201.81
4,2013,IE,106,IE01,106,1,6,2,280.22,0.00,1,1,1,280.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4917,2013,IE,85348,IE02,85348,1,6,1,1201.81,0.00,1,2,1,1201.81
4918,2013,IE,85379,IE02,85379,3,5,1,645.14,0.00,1,2,1,645.14
4919,2013,IE,85440,IE02,85440,2,5,1,288.64,405.34,1,1,3,693.98
4920,2013,IE,85453,IE02,85453,2,6,1,314.31,0.00,1,2,6,314.31


#Italy:

In [171]:
data_IT1 = pd.read_csv(path_to_data + "IT_Data_Pt1.csv", sep=",", header=0)
data_IT1 = data_IT1[data_IT1['DB010'].notnull()]

data_IT2 = pd.read_csv(path_to_data + "IT_Data_Pt2.csv", sep=",", header=0)
data_IT2 = data_IT2[data_IT2['DB010'].notnull()]
data_IT2 = data_IT2[data_IT2['DB010'] != 0]

data_IT = pd.concat([data_IT1, data_IT2])
data_IT.reset_index()
data = data_IT
data = data[data['DB010'].notnull()]
data['HH070'] = data['HH070'].replace(np.nan, 0)
data['HH071'] = data['HH071'].replace(np.nan, 0)
data['HH070+']=data['HH070']+data['HH071']
data_IT = data
data_IT


,DB010,DB020,DB030,DB040,HB030,HH021,HH030,HH050,HH070,HH071,HS050,HS060,HS120,HH070+
0,2013.0,IT,82.0,ITC1,82.0,1.0,5.0,1.0,233.0,0.0,1.0,1.0,4.0,233.0
1,2013.0,IT,103.0,ITC1,103.0,1.0,5.0,1.0,388.0,0.0,1.0,1.0,4.0,388.0
2,2013.0,IT,137.0,ITC1,137.0,5.0,5.0,1.0,198.0,346.0,1.0,1.0,3.0,544.0
3,2013.0,IT,163.0,ITC1,163.0,1.0,4.0,1.0,273.0,0.0,1.0,1.0,2.0,273.0
4,2013.0,IT,183.0,ITC1,183.0,3.0,3.0,1.0,771.0,0.0,1.0,2.0,1.0,771.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8482,2013.0,IT,510956.0,ITI4,510956.0,1.0,5.0,1.0,82.0,0.0,1.0,1.0,3.0,82.0
8483,2013.0,IT,511021.0,ITI4,511021.0,1.0,5.0,1.0,321.0,0.0,1.0,1.0,6.0,321.0
8484,2013.0,IT,511059.0,ITI4,511059.0,1.0,2.0,1.0,254.0,0.0,1.0,1.0,3.0,254.0
8485,2013.0,IT,511078.0,ITI4,511078.0,3.0,5.0,1.0,243.0,0.0,3.0,NaN,NaN,243.0


##Lithuania:

In [172]:
data_LT = pd.read_csv(path_to_data + "LT_Data.csv", sep=",", header=0)
data = data_LT
data = data[data['DB010'].notnull()]
data['HH070'] = data['HH070'].replace(np.nan, 0)
data['HH071'] = data['HH071'].replace(np.nan, 0)
data['HH070+']=data['HH070']+data['HH071']
data_LT = data
data_LT

,DB010,DB020,DB030,DB040,HB030,HH021,HH030,HH050,HH070,HH071,HS050,HS060,HS120,HH070+
0,2013,LT,7,LT00,7,1,5.0,1,79.45,0.00,2,1,2,79.45
1,2013,LT,54,LT00,54,1,3.0,2,115.85,0.00,1,1,4,115.85
2,2013,LT,58,LT00,58,1,4.0,1,139.50,0.00,1,2,3,139.50
3,2013,LT,88,LT00,88,2,4.0,1,328.04,96.54,1,1,2,424.58
4,2013,LT,104,LT00,104,1,4.0,1,161.27,0.00,1,2,4,161.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5137,2013,LT,130113,LT00,130113,1,4.0,1,181.01,0.00,1,2,4,181.01
5138,2013,LT,130279,LT00,130279,4,1.0,1,145.17,144.81,2,1,4,289.98
5139,2013,LT,130294,LT00,130294,1,1.0,1,150.60,0.00,1,1,4,150.60
5140,2013,LT,130373,LT00,130373,1,4.0,1,181.01,0.00,1,1,2,181.01


##Luxembourg:

In [173]:
data_LU = pd.read_csv(path_to_data + "LU_Data.csv", sep=",", header=0)
data = data_LU
data = data[data['DB010'].notnull()]
data['HH070'] = data['HH070'].replace(np.nan, 0)
data['HH071'] = data['HH071'].replace(np.nan, 0)
data['HH070+']=data['HH070']+data['HH071']
data_LU = data
data_LU

,DB010,DB020,DB030,DB040,HB030,HH021,HH030,HH050,HH070,HH071,HS050,HS060,HS120,HH070+
0,2013,LU,3,LU00,3,1,5.0,1,310,0.0,1,1.0,5,310.0
1,2013,LU,16,LU00,16,1,4.0,1,807,0.0,1,1.0,6,807.0
2,2013,LU,27,LU00,27,1,5.0,1,200,0.0,1,1.0,5,200.0
3,2013,LU,31,LU00,31,1,4.0,1,228,0.0,1,1.0,1,228.0
4,2013,LU,35,LU00,35,1,6.0,1,283,787.0,1,1.0,5,1070.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3765,2013,LU,40975,LU00,40975,1,5.0,1,225,0.0,1,1.0,6,225.0
3766,2013,LU,40979,LU00,40979,3,6.0,2,249,3000.0,1,2.0,6,3249.0
3767,2013,LU,41031,LU00,41031,1,6.0,1,232,0.0,1,1.0,5,232.0
3768,2013,LU,41032,LU00,41032,1,6.0,1,193,0.0,1,1.0,6,193.0


##Latvia:

In [174]:
data_LV = pd.read_csv(path_to_data + "LV_Data.csv", sep=",", header=0)
data = data_LV
data = data[data['DB010'].notnull()]
data['HH070'] = data['HH070'].replace(np.nan, 0)
data['HH071'] = data['HH071'].replace(np.nan, 0)
data['HH070+']=data['HH070']+data['HH071']
data_LV = data
data_LV

,DB010,DB020,DB030,DB040,HB030,HH021,HH030,HH050,HH070,HH071,HS050,HS060,HS120,HH070+
0,2013,LV,5,LV00,5,1,3.0,1,71.14,0.00,2,2,2,71.14
1,2013,LV,17,LV00,17,2,3.0,2,192.09,0.00,1,1,4,192.09
2,2013,LV,34,LV00,34,1,5.0,1,99.60,0.00,1,2,1,99.60
3,2013,LV,36,LV00,36,1,1.0,1,56.91,0.00,2,2,3,56.91
4,2013,LV,42,LV00,42,3,3.0,1,426.86,301.83,1,1,4,728.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6304,2013,LV,83007,LV00,83007,2,6.0,1,193.98,0.00,2,2,4,193.98
6305,2013,LV,83012,LV00,83012,1,6.0,1,193.98,133.28,2,1,2,327.26
6306,2013,LV,83027,LV00,83027,2,4.0,1,209.87,0.00,1,2,1,209.87
6307,2013,LV,83040,LV00,83040,3,3.0,1,213.43,0.00,2,1,4,213.43


##Malta:

In [175]:
data_MT = pd.read_csv(path_to_data + "MT_Data.csv", sep=",", header=0)
data = data_MT
data = data[data['DB010'].notnull()]
data['HH070'] = data['HH070'].replace(np.nan, 0)
data['HH071'] = data['HH071'].replace(np.nan, 0)
data['HH070+']=data['HH070']+data['HH071']
data_MT = data
data_MT

,DB010,DB020,DB030,DB040,HB030,HH021,HH030,HH050,HH070,HH071,HS050,HS060,HS120,HH070+
0,2013,MT,2,MT00,2,4,5,1,467.20,0.0,1,1.0,4,467.20
1,2013,MT,12,MT00,12,1,4,1,24.18,0.0,1,2.0,1,24.18
2,2013,MT,20,MT00,20,1,5,1,72.96,0.0,1,2.0,1,72.96
3,2013,MT,24,MT00,24,1,6,2,167.17,0.0,1,1.0,3,167.17
4,2013,MT,26,MT00,26,1,6,1,72.98,0.0,1,1.0,3,72.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4376,2013,MT,31186,MT00,31186,1,5,1,319.44,0.0,1,1.0,5,319.44
4377,2013,MT,31193,MT00,31193,3,6,2,84.52,0.0,1,2.0,2,84.52
4378,2013,MT,31195,MT00,31195,1,6,1,137.01,0.0,1,1.0,3,137.01
4379,2013,MT,31213,MT00,31213,1,6,1,362.96,0.0,1,1.0,4,362.96


##Netherlands:

In [176]:
data_NL = pd.read_csv(path_to_data + "NL_Data.csv", sep=",", header=0)
data = data_NL
data = data[data['DB010'].notnull()]
data['HH070'] = data['HH070'].replace(np.nan, 0)
data['HH071'] = data['HH071'].replace(np.nan, 0)
data['HH070+']=data['HH070']+data['HH071']
data_NL = data
data_NL

,DB010,DB020,DB030,DB040,HB030,HH021,HH030,HH050,HH070,HH071,HS050,HS060,HS120,HH070+
0,2013,NL,15,NaN,15,2.0,6.0,1.0,1169.0,0.0,1.0,1.0,4.0,1169.0
1,2013,NL,26,NaN,26,3.0,4.0,1.0,792.0,0.0,1.0,2.0,3.0,792.0
2,2013,NL,83,NaN,83,2.0,5.0,2.0,605.0,0.0,1.0,1.0,3.0,605.0
3,2013,NL,124,NaN,124,2.0,5.0,1.0,977.0,0.0,1.0,1.0,3.0,977.0
4,2013,NL,146,NaN,146,3.0,6.0,1.0,358.0,0.0,1.0,1.0,5.0,358.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10126,2013,NL,151346,NaN,151346,3.0,6.0,1.0,366.0,0.0,1.0,1.0,5.0,366.0
10127,2013,NL,151353,NaN,151353,1.0,5.0,1.0,954.0,0.0,1.0,1.0,4.0,954.0
10128,2013,NL,151357,NaN,151357,3.0,4.0,1.0,472.0,0.0,1.0,1.0,4.0,472.0
10129,2013,NL,151380,NaN,151380,3.0,5.0,1.0,297.0,0.0,1.0,2.0,5.0,297.0


##Romania:

In [177]:
data_RO = pd.read_csv(path_to_data + "RO_Data.csv", sep=",", header=0)
data = data_RO
data = data[data['DB010'].notnull()]
data['HH070'] = data['HH070'].replace(np.nan, 0)
data['HH071'] = data['HH071'].replace(np.nan, 0)
data['HH070+']=data['HH070']+data['HH071']
data_RO = data
data_RO

,DB010,DB020,DB030,DB040,HB030,HH021,HH030,HH050,HH070,HH071,HS050,HS060,HS120,HH070+
0,2013,RO,41,RO11,41,1,4,1,100.91,0.0,1,1,4,100.91
1,2013,RO,52,RO11,52,5,4,1,56.06,0.0,1,1,3,56.06
2,2013,RO,68,RO11,68,1,4,1,100.91,0.0,1,1,2,100.91
3,2013,RO,81,RO11,81,1,5,1,11.21,0.0,1,1,4,11.21
4,2013,RO,92,RO11,92,1,3,1,123.34,0.0,1,2,1,123.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7555,2013,RO,140322,RO42,140322,3,2,2,78.49,0.0,2,2,1,78.49
7556,2013,RO,140353,RO42,140353,1,3,1,22.43,0.0,1,1,1,22.43
7557,2013,RO,140364,RO42,140364,5,2,1,100.91,0.0,1,1,4,100.91
7558,2013,RO,140369,RO42,140369,1,3,1,100.91,0.0,2,2,1,100.91


##Sweden:

In [178]:
data_SE = pd.read_csv(path_to_data + "SE_Data.csv", sep=",", header=0)
data = data_SE
data = data[data['DB010'].notnull()]
data['HH070'] = data['HH070'].replace(np.nan, 0)
data['HH071'] = data['HH071'].replace(np.nan, 0)
data['HH070+']=data['HH070']+data['HH071']
data_SE = data
data_SE

,DB010,DB020,DB030,DB040,HB030,HH021,HH030,HH050,HH070,HH071,HS050,HS060,HS120,HH070+
0,2013,SE,39,SE11,39,2.0,5.0,1.0,944.27,0.00,1.0,1.0,6.0,944.27
1,2013,SE,49,SE11,49,2.0,3.0,1.0,899.35,1148.88,1.0,1.0,4.0,2048.23
2,2013,SE,65,SE11,65,2.0,5.0,1.0,736.43,0.00,1.0,1.0,4.0,736.43
3,2013,SE,77,SE11,77,2.0,5.0,1.0,1253.43,45.96,1.0,2.0,6.0,1299.39
4,2013,SE,88,SE11,88,2.0,4.0,1.0,790.32,0.00,1.0,1.0,6.0,790.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6196,2013,SE,92654,SE33,92654,2.0,5.0,1.0,1108.79,0.00,1.0,1.0,4.0,1108.79
6197,2013,SE,92655,SE33,92655,2.0,4.0,1.0,987.70,0.00,1.0,1.0,6.0,987.70
6198,2013,SE,92659,SE33,92659,2.0,4.0,1.0,1756.41,0.00,1.0,1.0,6.0,1756.41
6199,2013,SE,92684,SE33,92684,2.0,5.0,1.0,781.47,0.00,1.0,1.0,6.0,781.47


##Slovenia:

In [179]:
data_SI = pd.read_csv(path_to_data + "SI_Data.csv", sep=",", header=0)
data = data_SI
data = data[data['DB010'].notnull()]
data['HH070'] = data['HH070'].replace(np.nan, 0)
data['HH071'] = data['HH071'].replace(np.nan, 0)
data['HH070+']=data['HH070']+data['HH071']
data_SI = data
data_SI

,DB010,DB020,DB030,DB040,HB030,HH021,HH030,HH050,HH070,HH071,HS050,HS060,HS120,HH070+
0,2013,SI,486,NaN,486,1,3,2,89.17,0.0,1,2,3,89.17
1,2013,SI,611,NaN,611,1,3,1,402.42,0.0,1,2,1,402.42
2,2013,SI,818,NaN,818,4,4,1,301.26,0.0,1,2,5,301.26
3,2013,SI,971,NaN,971,1,5,1,473.00,0.0,1,2,5,473.00
4,2013,SI,1085,NaN,1085,1,4,1,105.17,0.0,2,2,4,105.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8982,2013,SI,803675,NaN,803675,2,6,1,201.67,0.0,1,2,2,201.67
8983,2013,SI,803680,NaN,803680,5,3,1,267.75,0.0,1,2,2,267.75
8984,2013,SI,803698,NaN,803698,1,3,1,124.75,0.0,1,1,1,124.75
8985,2013,SI,803718,NaN,803718,5,4,1,108.00,0.0,1,2,2,108.00


##Slovakia:

In [180]:
data_SK = pd.read_csv(path_to_data + "SK_Data.csv", sep=",", header=0)
data = data_SK
data = data[data['DB010'].notnull()]
data['HH070'] = data['HH070'].replace(np.nan, 0)
data['HH071'] = data['HH071'].replace(np.nan, 0)
data['HH070+']=data['HH070']+data['HH071']
data_SK = data
data_SK

,DB010,DB020,DB030,DB040,HB030,HH021,HH030,HH050,HH070,HH071,HS050,HS060,HS120,HH070+
0,2013,SK,32,SK01,32,5.0,2.0,1.0,148.78,0.00,1.0,2.0,3.0,148.78
1,2013,SK,41,SK01,41,3.0,3.0,1.0,175.69,0.00,1.0,1.0,3.0,175.69
2,2013,SK,49,SK01,49,3.0,4.0,1.0,101.17,89.65,1.0,1.0,2.0,190.82
3,2013,SK,56,SK01,56,3.0,2.0,1.0,108.27,0.00,1.0,2.0,3.0,108.27
4,2013,SK,61,SK01,61,1.0,2.0,1.0,183.25,0.00,2.0,2.0,2.0,183.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5397,2013,SK,92495,SK04,92495,NaN,NaN,NaN,0.00,0.00,NaN,NaN,NaN,0.00
5398,2013,SK,92506,SK04,92506,NaN,NaN,NaN,0.00,0.00,NaN,NaN,NaN,0.00
5399,2013,SK,92536,SK04,92536,NaN,NaN,NaN,0.00,0.00,NaN,NaN,NaN,0.00
5400,2013,SK,92550,SK04,92550,NaN,NaN,NaN,0.00,0.00,NaN,NaN,NaN,0.00


##United Kingdom:

In [181]:
data_UK = pd.read_csv(path_to_data + "UK_Data.csv", sep=",", header=0)
data = data_UK
data = data[data['DB010'].notnull()]
data['HH070'] = data['HH070'].replace(np.nan, 0)
data['HH071'] = data['HH071'].replace(np.nan, 0)
data['HH070+']=data['HH070']+data['HH071']
data_UK = data
data_UK

,DB010,DB020,DB030,DB040,HB030,HH021,HH030,HH050,HH070,HH071,HS050,HS060,HS120,HH070+
0,2013,UK,31,UKC1,31,1,5.0,1.0,150,0.0,2.0,1.0,3.0,150.0
1,2013,UK,56,UKC1,56,4,5.0,1.0,250,0.0,1.0,1.0,5.0,250.0
2,2013,UK,177,UKC1,177,3,4.0,1.0,200,0.0,1.0,2.0,3.0,200.0
3,2013,UK,312,UKC1,312,3,6.0,1.0,1550,0.0,1.0,1.0,5.0,1550.0
4,2013,UK,333,UKC1,333,1,4.0,1.0,850,0.0,1.0,1.0,2.0,850.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10169,2013,UK,539994,UKN0,539994,1,5.0,1.0,600,200.0,1.0,2.0,4.0,800.0
10170,2013,UK,540001,UKN0,540001,2,5.0,1.0,1650,0.0,1.0,2.0,4.0,1650.0
10171,2013,UK,540005,UKN0,540005,3,2.0,1.0,1350,600.0,1.0,1.0,2.0,1950.0
10172,2013,UK,540009,UKN0,540009,4,5.0,1.0,1200,0.0,2.0,2.0,3.0,1200.0
